https://campus.datacamp.com/courses/hypothesis-testing-in-r/

- korrelation med cor()
- calculation z-scores and p-values
- confidence intervals
- chisquare med chisq.test (https://www.tutorialspoint.com/r/r_chi_square_tests.htm)
- ttest

https://infer.netlify.app/reference/index.html

https://rdrr.io/r/stats/prop.test.html

In [ ]:
late_prop_hyp <- 0.06

# Calculate the standard error
std_error <- late_shipments_boot_distn %>% 
  summarize(sd_late_prop = sd(late_prop)) %>% 
  pull(sd_late_prop)

# Find z-score of late_prop_samp
z_score <- (late_prop_samp - late_prop_hyp) / std_error

# See the results
z_score

In [ ]:
## Right tail

# Calculate the z-score of late_prop_samp
z_score <- (late_prop_samp - late_prop_hyp) / std_error

# Calculate the p-value
p_value <- pnorm(z_score, lower.tail = FALSE)
                 
# See the result
p_value    

In [ ]:
# Calculate 95% confidence interval using quantile method
conf_int_quantile <- late_shipments_boot_distn %>%
  summarize(
    lower = quantile(prop_late_shipments, 0.025),
    upper = quantile(prop_late_shipments, 0.975)
  )

# See the result
conf_int_quantile

In [ ]:
## Two sample mean test statistic by hand
# Calculate the numerator of the test statistic
numerator <- xbar_no - xbar_yes # means for the two groups
 
# Calculate the denominator of the test statistic
denominator <- sqrt(s_no ^ 2 / n_no + s_yes ^ 2 / n_yes) # s for sd n for sample size

# Calculate the test statistic
t_stat <- numerator / denominator

# See the result
t_stat

# Calculate the degrees of freedom
degrees_of_freedom <- n_no + n_yes - 2 # n - number of observations

# Calculate the p-value from the test stat
p_value <- pt(t_stat, df = degrees_of_freedom)

# See the result
p_value

In [ ]:
# Conduct a t-test on diff
test_results <- t.test(sample_dem_data$diff, alternative = "greater") # greater - right tail, lesser - left tail. diff-column differences in between two measures

# See the results
test_results

In [ ]:
# Conduct a paired t-test on dem_percent_12 and dem_percent_16
test_results <- t.test(
  x = sample_dem_data$dem_percent_12,
  y = sample_dem_data$dem_percent_16,
  paired = TRUE,
  alternative = "greater"
)

# See the results
test_results

In [ ]:
# From previous step
mdl_pack_price_vs_shipment_mode <- lm(pack_price ~ shipment_mode, data = late_shipments)

# Perform ANOVA on the regression model
anova(mdl_pack_price_vs_shipment_mode)

In [ ]:
# proportion tests

# Hypothesize that the proportion of late shipments is 6%
p_0 <- 0.06

# Calculate the sample proportion of late shipments
p_hat <- late_shipments %>%
  summarize(prop_late = mean(late == "Yes")) %>%
  pull(prop_late)

# Calculate the sample size
n <- nrow(late_shipments)

# From previous step
p_0 <- 0.06
p_hat <- late_shipments %>%
  summarize(prop_late = mean(late == "Yes")) %>%
  pull(prop_late)
n <- nrow(late_shipments)

# Calculate the numerator of the test statistic
numerator <- p_hat - p_0

# Calculate the denominator of the test statistic
denominator <- sqrt(p_0 * (1 - p_0) / n)

# Calculate the test statistic
z_score <- numerator / denominator

# See the result
z_score

# From previous step
p_0 <- 0.06
p_hat <- late_shipments %>%
  summarize(prop_late = mean(late == "Yes")) %>%
  pull(prop_late)
n <- nrow(late_shipments)
numerator <- p_hat - p_0
denominator <- sqrt(p_0 * (1 - p_0) / n)
z_score <- numerator / denominator

# Calculate the p-value from the z-score
p_value <- pnorm(z_score, lower.tail = FALSE)

# See the result
p_value

In [ ]:
# Perform a chi-square test of independence on
# freight_cost_group and vendor_inco_term
test_results <- late_shipments %>% 
  chisq_test(freight_cost_group ~ vendor_inco_term)

# See the results
test_results

In [ ]:
## simulation based hypothesis testing - proportions

# From previous steps
null_distn <- late_shipments %>% 
  specify(
    late ~ freight_cost_group, 
    success = "Yes"
  ) %>% 
  hypothesize(null = "independence") %>% 
  generate(reps = 2000, type = "permute") %>% 
  calculate(
    stat = "diff in props", 
    order = c("expensive", "reasonable")
  )
obs_stat <- late_shipments %>% 
  specify(
    late ~ freight_cost_group, 
    success = "Yes"
  ) %>% 
  calculate(
    stat = "diff in props", 
    order = c("expensive", "reasonable")
  )

# Visualize the null dist'n, adding a vertical line at
# the observed statistic
visualize(null_distn) +
  geom_vline(aes(xintercept = stat), data = obs_stat)

# Get the p-value
p_value <- get_p_value(
  null_distn, obs_stat, 
  direction = "greater"
)

# See the result
p_value

In [ ]:
## simulation based hypothesis testing - means

# From previous steps
null_distn <- late_shipments %>% 
  specify(weight_kilograms ~ late) %>% 
  hypothesize(null = "independence") %>% 
  generate(reps = 1000, type = "permute") %>% 
  calculate(stat = "diff in means", order = c("No", "Yes"))
obs_stat <- late_shipments %>% 
  specify(weight_kilograms ~ late) %>% 
  calculate(stat = "diff in means", order = c("No", "Yes"))

# Get the p-value
p_value <- get_p_value(
  null_distn, obs_stat,
  direction = "less"
)

# See the result
p_value